## In this notebook I will try to experiment with connecting all my morphology works

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('Libraries/')

from imports import *
from utils import *

main_path = '/cortexdisk/data2/NestorRembrandtCollab/'
code_dir = os.path.join(main_path,'Code Repositories')
sys.path.append(code_dir)

data_repository = 'atlas_files'

mesoscale_path = '../mesoscale_extractor/'
sys.path.append(mesoscale_path)

from morphometrics_tools import *
from morpho_embedding_tools import *
from topography_tools import *
from projection_utils import *
from mesoscale_extractor import NeuronPopulation

### Initialize a number of important variables


In [2]:
res = 10
useful_vars = load_useful_data(data_repository)
[annotation, template, acr2id, id2acr, ancestorsById, neuriteLengthDistribution, acr_to_morpho_id] = useful_vars
braintell_dir = os.path.join(main_path,'Data Repositories/Braintell')
mouselight_dir = os.path.join(main_path,'Data Repositories/Mouselight/json')
nld_list = neuriteLengthDistribution.keys()

## Part III: Highlighting projection motifs

#### Warning:   
If I want to load exactly the data I used for the paper, I will need to change this one with the old acr2morpho_id file found at: /cortexdisk/data2/NestorRembrandtCollab/Data Repositories/mouse_connectivity/acr_to_morpho_id.json

In [ ]:
layers = ['1','2/3','4','5','6a','6b']
source_areas = ['VPM'] 
target_areas = ['SSs', 'SSp-bfd', 'SSp-m', 'SSp-ul', 'SSp-ll', 'SSp-n']

neuropop_cls = NeuronPopulation(data_path = data_repository, res = res)

mesoscale_stats_df = neuropop_cls.make_connectivity_matrix(source_areas, target_areas, 
                                                            feature = 'counts', mode = 'full', extract = 'terminals')

## Keeping the order constant as done in the paper
mesoscale_stats_df = mesoscale_stats_df.reindex(['SSp-ul', 'SSp-m', 'SSp-n', 'SSp-ll', 'SSs', 'SSp-bfd'], 
                                                axis = 1)

mesoscale_stats_array = np.array(mesoscale_stats_df)
mesoscale_stats_array[np.where(np.isnan(mesoscale_stats_array))] = 0

##### Repeat t-SNE visualizations with both clustering results, Rembrandt style labeling, and motif labeling

In [ ]:
cmap = ['#00FF00','#FF0000','#DD8800','#0000FF','#00AAFF','#333333','#880000','#884400','#000088','#0088DD',"#00FF00"]    
Proj_mat_embedded, max_proj_labels = get_tSNE(mesoscale_stats_df, cmap)

In [ ]:
unique_motifs, representation_list = get_motif_distribution(mesoscale_stats_df, proj_thr = 5, source_area = 'VPM')

In [ ]:
motif_count_list = get_motif_counts(mesoscale_stats_df)

### 06/12/2022: Here I will experiment with making flatmaps out of the clusters

In [ ]:
another_data_repository = os.path.join(main_path,'Data Repositories/mouse_connectivity/')
flatmap_dir = os.path.join(main_path,'Data Repositories/') 
source_vol, target_flts, color_neuron_dict = flatmap_the_clusters(mesoscale_stats_df, neuropop_cls, max_proj_labels, 
                                                                  annotation, template, acr2id, another_data_repository,
                                                                  flatmap_dir, code_dir, source = 'VPM')

### CPD Strategy: utilize the affinity matrix to assess morphological clusters 

In [ ]:
source_neuron_ids = sorted(list(mesoscale_stats_df.loc['VPM'].index))
source_neuron_ids = [name_2_fd(val, acr_to_morpho_id, 'VPM', mouselight_dir, braintell_dir) for val in source_neuron_ids]
# comp_res = compare_source_to_targets(source_neuron_ids[10:50], source_neuron_ids[10:50]) # or ...
comp_res = compare_source_to_targets(source_neuron_ids, source_neuron_ids) 
Affinity_df_total, Affinity_df_trs_total, soma_dst_df, soma_pos = comp_res

In [ ]:
keyRegions = list(mesoscale_stats_df.columns)
morpho_res = morphological_embedding(Affinity_df_trs_total, soma_pos, annotation, color_neuron_dict, 
                                     max_proj_labels, nld_list, keyRegions)
Affinity_mat_red, diff_grad, diff_grad_nrm, neuronid_to_grad, mdist_clust_labels = morpho_res

In [ ]:
flatmap_the_gradients(neuropop_cls, neuronid_to_grad, annotation, template, acr2id, nld_list, another_data_repository, 
                      flatmap_dir, code_dir)

In [ ]:
motif_to_grad2_df, new_df3 = get_motif_barplots(mesoscale_stats_df, neuronid_to_grad, nld_list, participation_thr = 4, 
                                                exclude_list = ['SSp-ul','SSp-ll','SSp-tr'])

### Topographic comparisons with source/target position and alignment with morphological gradient

In [ ]:
soma_arr, target_arr, grad_position = get_source_and_target_arrays(neuronid_to_grad, neuropop_cls, nld_list, 
                                                                   annotation, id2acr, 'SSp')
Affine_grand, X_trs_aff, r, Rotats_grand, mse_aff = get_topopgraphic_rotations(soma_arr, target_arr)
get_topographic_plots(soma_arr, target_arr, X_trs_aff)

In [ ]:
topograd_res = align_topography_and_gradient(soma_arr, target_arr, grad_position)
lateromed_grad_src, anterolat_grad_trg, rho1, pval1, rho2, pval2 = topograd_res

### Morphometric comparisons between the cell clusters

In [ ]:
morpho_res = get_morphometrics_per_cluster(Affinity_df_trs_total, neuropop_cls, mdist_clust_labels, 
                                           nld_list, mouselight_dir, braintell_dir)
morpho_stats_per_cluster, morpho_stats_per_cluster2, list_of_neurons_per_cluster, keep_list = morpho_res
plot_morphometrics(morpho_stats_per_cluster, morpho_stats_per_cluster2, list_of_neurons_per_cluster, neuronid_to_grad,
                   keep_list, mdist_clust_labels)

### Calling the Han et al code for estimating the over/underrepresentation of motifs

In [ ]:
area_labels = list(mesoscale_stats_df.columns)
alpha = 0.05
proj_thr = 6
results = Port_Han_Motif_Code(mesoscale_stats_array, proj_thr, alpha, area_labels)
motif,pval_min,significance,overorunder,observed,expected,p_all,neuronnum, pvalue = results

In [ ]:
layers = ['1','2/3','4','5','6a','6b']
target_areas_layered = [val1+val2 for val1 in target_areas for val2 in layers]
mesoscale_stats_df_layered = neuropop_cls.make_connectivity_matrix(source_areas, target_areas_layered, 
                                                            feature = 'length', mode = 'full', extract = 'terminals')

### Obtaining the layer-specific patterns associated with multifocal projections

In [ ]:
proj_trg_laminar_dict = get_laminar_motifs(mesoscale_stats_df, mesoscale_stats_df_layered)